In [ ]:
import GPy
import numpy as np

# Example data
X_train = np.random.uniform(-3.,3.,(50,1)) # Training inputs
Y_train = np.sin(X_train) + np.random.randn(50,1)*0.05 # Training outputs

# Example of base models' predictions for ensemble (simulated here as random)
f_k = np.random.randn(50, 1)




# Define the zero mean function for the GP
mean_function = GPy.mappings.Constant(input_dim=1, output_dim=1, value=0)

# Define the kernel function with Radial Basis Function (RBF)
# The variance and lengthscale are hyperparameters and should be chosen according to the problem at hand.
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)

# Create the Gaussian Process model
gp_model = GPy.models.GPRegression(X_train, Y_train - f_k, kernel, mean_function=mean_function)

# Fit the GP model to the residuals
gp_model.optimize(messages=True)

# Predict using the GP model to get the residual process
X_test = np.linspace(-5, 5, 100).reshape(-1, 1)  # Test inputs
mean, variance = gp_model.predict(X_test)  # Residual process predictions

# Final prediction considering the ensemble and the residual process
ensemble_prediction = f_k.mean() + mean  # Assuming f_k.mean() as a dummy ensemble prediction

# The complete model prediction would be the sum of the ensemble predictions and the GP residuals.


In [ ]:
# calibration function
import GPy
import numpy as np

# Simulated observed data
X_observed = np.random.uniform(-3., 3., (50, 1))  # Observed input locations
Y_observed = np.random.uniform(0, 1, (50, 1))  # Observed CDF values (should be between 0 and 1)

# Define the kernel function - Matérn 3/2 kernel
kernel_G = GPy.kern.Matern32(input_dim=1, variance=1., lengthscale=1.)

# Since we want G to be a CDF, the mean function should be identity, but GPy does not have this functionality out of the box.
# So we use a zero mean function, and we will need to enforce monotonicity through other means, such as post-processing or constraints during optimization.
mean_function_G = GPy.mappings.Linear(input_dim=1, output_dim=1)
mean_function_G.C = np.array([[1]])  # Identity mapping
mean_function_G.bias = np.array([[0]])  # No bias

# Create the Gaussian Process model for G
gp_calibration = GPy.models.GPRegression(X_observed, Y_observed, kernel_G, mean_function=mean_function_G)

# Optimization constraints would be added here to ensure the output is monotonically increasing and bounded between 0 and 1.

# Fit the GP model to the observed CDF data
gp_calibration.optimize(messages=True)

# Example use: Calibrating a new set of predictions
X_new = np.linspace(-5, 5, 100).reshape(-1, 1)  # New input locations
Phi_e_new = np.random.uniform(0, 1, (100, 1))  # New predictions for CDF values from the base ensemble model

# Use the GP model to calibrate the new predictions
G_Phi_e_new, _ = gp_calibration.predict(X_new)

# The calibrated predictions would then be G(Phi_e_new), which you'd get by passing Phi_e_new through the calibration model G.
# Since our GP model does not directly apply to this form, one would need to adjust the predictions manually or alter the GP setup to support this functionality.
